<a href="https://cognitiveclass.ai"><img src = "https://ibm.box.com/shared/static/9gegpsmnsoo25ikkbl4qzlvlyjbgxs5x.png" width = 400 align="left"> </a>

<h1 align=center><font size = 5>IBM Data Science Capstone - Eugene Bible</font></h1>

## Introduction

This is the Jupyter Notebook to be used for the IBM Data Science Professional Certificate course by Coursera. 

In this Jupyter notebook, neighborhoods in the city of Toronto will be explored, segmented, and clustered. 
First, the data will be scraped from the Wikipedia page and be wrangled, cleaned, and then read into a pandas dataframe in a structured format. Once structured, the neighborhoods in Toronoto will be explored and clustered.

Author: Eugene Bible

## Table of Contents

1. <a href="#item1">Segmenting and Clustering Neighborhoods in Toronto</a>
2. <a href="#item2">The Battle of Neighborhoods - Part 1</a>  
3. <a href="#item3">The Battle of Neighborhoods - Part 2</a>  

This notebook will be used for the projects from the Coursera IBM Data Science Capstone course. 

In [2]:
import pandas as pd
import numpy as np

In [3]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!
